In [0]:
!mkdir './data'
!unzip '/content/drive/My Drive/Steering_Datasets_Rev1/Simulator_Dataset/IMG6.zip' -d '/content/data'

In [0]:
import torch 
from torchvision import transforms, utils 
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd 

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import torchvision

from torch.utils.data.sampler import SubsetRandomSampler

import torchvision.models as models 
import torch.nn as nn 

import torch.optim as optim 
import cv2
import random

In [0]:
class SteeringDataset(Dataset):
    def __init__(self, image_path, label_path, data_transforms=None):
        """
        Your code here
        Hint: Use the python csv library to parse labels.csv
        """
        self.image_path = image_path
        self.label_path = label_path
        
        # Training Dataset
        self.name = pd.read_csv(self.label_path + 'driving_log.csv', usecols=range(0,1))
        self.labels = pd.read_csv(self.label_path + 'driving_log.csv', usecols=range(3,4))
        self.center_data = pd.concat([self.name, self.labels], axis=1) #combine image name and label dataframes
        
        # Training Dataset Only!!!
        # self.center_data = self.center_data[self.center_data["filename"].str.contains('center')] # only keep center image names and labels
        self.name = pd.DataFrame(self.center_data[self.center_data.columns[0]]) # center images names
        self.len = self.name.shape[0]
        self.labels = pd.DataFrame(self.center_data[self.center_data.columns[1]]) # center image labels

    def __len__(self):
        """
        Your code here
        """
        return self.len

    def __getitem__(self, idx):
        """
        Your code here
        return a tuple: img, label
        """
        # For Training Dataset
        img = Image.open(self.image_path + str(self.name.iloc[idx, 0])[71:]) # [7:] removes 'center/' before the image number since the zipped images just have numbers
        
        img = img.crop((0, 60, 320, 160))  # crop image (remove above horizon). 
        img = img.resize((224,224)) #resize image --> pretrained resnet model needs img sizes of 224 x 224

        # img = np.array(img)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # img = cv2.Canny(img, 100, 300)
        # transform = transforms.ToTensor()
        # transform = transforms.Compose([
        #     transforms.ToTensor(),
        #     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        # ])
        transform = transforms.ToTensor()
        
        # img = torch.cat((img,img,img),0)
        label = self.labels.iloc[idx][0]

        if random.uniform(0,1) < 0.5:
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
            label = -label

        img = transform(img)
        return img, label

In [0]:
def train_test_split(dataset, batch_size = 16, validation_split = .2, shuffle_dataset = True):
  size = len(dataset) 
  idx = list(range(size))
  if shuffle_dataset: 
    np.random.seed(42)
    np.random.shuffle(idx)

  split = int(np.floor(validation_split * size))
  train_idx, validation_idx = idx[split:], idx[:split]

  # create data subsamplers
  train_sampler = SubsetRandomSampler(train_idx)
  validation_sampler = SubsetRandomSampler(validation_idx)

  # create data loaders 
  train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler) 
  validation_loader = DataLoader(dataset, batch_size=batch_size, sampler=validation_sampler)
  return train_loader, validation_loader

In [0]:
def load_data(image_path, label_path, data_transforms=None, num_workers=0, batch_size=1, shuffle=True):
    dataset = SteeringDataset(image_path, label_path, data_transforms)
    return DataLoader(dataset, num_workers=num_workers, batch_size=batch_size, shuffle=shuffle)

In [0]:
class ClassificationLoss(torch.nn.Module):
    def forward(self, input, target):
        m = nn.MSELoss()
        return torch.sqrt(m(input.view(input.size(0)),target))

In [0]:
def train(args, model, train_loader, valid_loader, model_save_name):
  model.train() 
  
  model.to(args.device)

  # define optimizer and loss 
  optimizer = optim.Adam(model.parameters(), lr=args.learning_rate) 
  criterion = ClassificationLoss() 
  
  criterion.to(args.device)

  train_losses = [] 
  loss_list = [] 
  for epoch in range(1,args.num_epochs+1):
    ### 
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader): 
        
      data = data.to(args.device)
      target = target.to(args.device)
        
      optimizer.zero_grad() 
      output = model(data)
      loss = criterion(output,target.type(torch.FloatTensor).to(args.device))
      loss.backward() 
      optimizer.step() 

      loss_list.append(loss.item()) 
      if batch_idx % args.log_interval == 0: 
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, (batch_idx+1) * len(data), len(train_loader)*len(data),
                  100. * batch_idx / len(train_loader), loss.item()))
        
        train_losses.append(loss.item())
    test(model, valid_loader)
    print(f'\n Average loss for Epoch {epoch}: {sum(loss_list)/len(loss_list)} \n')
    del loss_list[:]
  torch.save(model.state_dict(), './' + model_save_name)
  return model

In [0]:
def test(model, test_loader): 

  # resnet.load_state_dict(torch.load(path_to_model))
  resnet = model
  resnet.eval() 
  resnet.to(args.device)

  predictions = []

  # define loss 
  criterion = ClassificationLoss() 
  test_loss = 0 
  with torch.no_grad(): 
    for batch_idx, (data, target) in enumerate(test_loader): 
      data = data.to(args.device)
      target = target.to(args.device)
      output = resnet(data)
      output = output.cpu()
      #print(output)
      predictions.append(np.array(output)) # for video maker  
      output = output.to(args.device)
      loss =  criterion(output, target).item() 
      test_loss += loss 
      if batch_idx % 100 == 0:
        print(f'Batch_idx: {batch_idx}   Average Loss So Far: {test_loss / (batch_idx+1)}')

    test_loss /= len(test_loader) 
    print('Overall test loss = ', test_loss)
    
  return

In [0]:
class Args(object):
  def __init__(self):
    self.learning_rate = 0.001 
    self.momentum = .5 
    self.num_epochs = 10

    self.batch_size = 16
    self.log_interval = 10
    self.val_size = 0.2
    self.shuffle = True
    
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
# Training Dataset Paths
label_path = '/content/data/IMG6/'
image_path = '/content/data/IMG6/'
# model_path = '.\model_t2_normal_4epoch.pt'

# Parameters
args = Args() 


# Model Selection: (Custom Resnet18)
resnet = models.resnet18(pretrained=False, progress=True)
for param in resnet.parameters():
    param.requires_grad = True 
resnet.fc = nn.Sequential(nn.Linear(512, 512),
                          nn.ReLU(),
                          nn.Linear(512, 1))

# resnet.load_state_dict(torch.load(model_path))

# Load & Train All Training Data
data_transforms = None
dataset = SteeringDataset(image_path, label_path, data_transforms)
train_loader, valid_loader = train_test_split(dataset, batch_size = 16, validation_split = .2, shuffle_dataset = True)

# dataset_loader = load_data(image_pa th, label_path, batch_size=args.batch_size, shuffle=args.shuffle)
train(args, model=resnet, train_loader=train_loader,valid_loader = valid_loader, model_save_name='model_V#.pt')


# Cross-Validation Loading / Training / Testing
'''
data = SteeringDataset(image_path, label_path)
train_loader, validation_loader = train_test_split(data, batch_size=args.batch_size, validation_split=args.val_size, shuffle_dataset=True)
model = train(args, model=resnet, train_loader=train_loader, model_save_name='model_V#_crossval')
test(model, validation_loader)
'''

Train Epoch: 1 [16/33440 (0%)]	Loss: 0.451647
Train Epoch: 1 [176/33440 (0%)]	Loss: 0.493698
Train Epoch: 1 [336/33440 (1%)]	Loss: 0.390299
Train Epoch: 1 [496/33440 (1%)]	Loss: 0.258651
Train Epoch: 1 [656/33440 (2%)]	Loss: 0.262615
Train Epoch: 1 [816/33440 (2%)]	Loss: 0.277835
Train Epoch: 1 [976/33440 (3%)]	Loss: 0.482847
Train Epoch: 1 [1136/33440 (3%)]	Loss: 0.356790
Train Epoch: 1 [1296/33440 (4%)]	Loss: 0.383300
Train Epoch: 1 [1456/33440 (4%)]	Loss: 0.199326
Train Epoch: 1 [1616/33440 (5%)]	Loss: 0.173882
Train Epoch: 1 [1776/33440 (5%)]	Loss: 0.366126
Train Epoch: 1 [1936/33440 (6%)]	Loss: 0.353485
Train Epoch: 1 [2096/33440 (6%)]	Loss: 0.388190
Train Epoch: 1 [2256/33440 (7%)]	Loss: 0.281698
Train Epoch: 1 [2416/33440 (7%)]	Loss: 0.226860
Train Epoch: 1 [2576/33440 (8%)]	Loss: 0.398291
Train Epoch: 1 [2736/33440 (8%)]	Loss: 0.188800
Train Epoch: 1 [2896/33440 (9%)]	Loss: 0.264018
Train Epoch: 1 [3056/33440 (9%)]	Loss: 0.232765
Train Epoch: 1 [3216/33440 (10%)]	Loss: 0.435016

"\ndata = SteeringDataset(image_path, label_path)\ntrain_loader, validation_loader = train_test_split(data, batch_size=args.batch_size, validation_split=args.val_size, shuffle_dataset=True)\nmodel = train(args, model=resnet, train_loader=train_loader, model_save_name='model_V#_crossval')\ntest(model, validation_loader)\n"